<a href="https://colab.research.google.com/github/NicholayKurbatov/AL_project/blob/master/demo_swin_doc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **SwinDocSegmenter**
This notebook try to demonstrate how to use this method efficiently for complex document layout analysis.

## **Step1: Create Environment and Install the necessary packages**

In [1]:
!git clone https://github.com/ayanban011/SwinDocSegmenter.git

Cloning into 'SwinDocSegmenter'...
remote: Enumerating objects: 568, done.
remote: Counting objects: 100% (299/299), done.
remote: Compressing objects: 100% (195/195), done.
remote: Total 568 (delta 113), reused 260 (delta 94), pack-reused 269 (from 1)
Receiving objects: 100% (568/568), 20.82 MiB | 13.83 MiB/s, done.
Resolving deltas: 100% (190/190), done.


In [2]:
%cd SwinDocSegmenter/

/content/SwinDocSegmenter


In [3]:
!python -m pip install pyyaml==5.1
# Detectron2 has not released pre-built binaries for the latest pytorch (https://github.com/facebookresearch/detectron2/issues/4053)
# so we install from source instead. This takes a few minutes.
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

# Install pre-built detectron2 that matches pytorch version, if released:
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
#!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/{CUDA_VERSION}/{TORCH_VERSION}/index.html

# exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 5.8 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-cpxcq1ac
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-cpxcq1ac
  Resolved https://github.com/facebookresearch/detectron2.git to commit 9604f5995cc628619f0e4fd913453b4d7d61db3f
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50

In [4]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.9/74.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [5]:
%cd maskdino/modeling/pixel_decoder/ops
!sh make.sh

/content/SwinDocSegmenter/maskdino/modeling/pixel_decoder/ops
running build
running build_py
creating build/lib.linux-x86_64-cpython-311/modules
copying modules/__init__.py -> build/lib.linux-x86_64-cpython-311/modules
copying modules/ms_deform_attn.py -> build/lib.linux-x86_64-cpython-311/modules
creating build/lib.linux-x86_64-cpython-311/functions
copying functions/__init__.py -> build/lib.linux-x86_64-cpython-311/functions
copying functions/ms_deform_attn_func.py -> build/lib.linux-x86_64-cpython-311/functions
running build_ext
/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:497: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:416: UserWarning: The detected CUDA version (12.5) has a minor version mismatch with the version that was used to com

In [6]:
%cd ../../../..

/content/SwinDocSegmenter


Ufff! This is the most critical job we have to done. As it is done. The inference is much easier now.

## **Step2: Inference on test dataset**

In [7]:
#!/usr/bin/env python
# Copyright (c) Facebook, Inc. and its affiliates.

import argparse
import json
import numpy as np
import os
from collections import defaultdict
import cv2
import tqdm

from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.structures import Boxes, BoxMode, Instances
from detectron2.utils.file_io import PathManager
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import Visualizer

from detectron2.data.datasets import register_coco_instances

In [8]:
register_coco_instances("prima_val",{},"/content/drive/MyDrive/layout-model-training-master/layout-model-training-master/tools/annotations-val.json","/content/drive/MyDrive/layout-model-training-master/layout-model-training-master/data/prima/val")
input = "/content/coco_instances_results.json"
output = "/content"
conf_threshold = 0.5

In [9]:
with PathManager.open(input, "r") as f:
  predictions = json.load(f)

pred_by_image = defaultdict(list)
for p in predictions:
  pred_by_image[p["image_id"]].append(p)

dicts = list(DatasetCatalog.get("prima_val"))
metadata = MetadataCatalog.get("prima_val")
if hasattr(metadata, "thing_dataset_id_to_contiguous_id"):
  def dataset_id_map(ds_id):
    return metadata.thing_dataset_id_to_contiguous_id[ds_id]

os.makedirs(output, exist_ok=True)

FileNotFoundError: [Errno 2] No such file or directory: '/content/coco_instances_results.json'

In [ ]:
def create_instances(predictions, image_size):
    ret = Instances(image_size)

    score = np.asarray([x["score"] for x in predictions])
    chosen = (score > conf_threshold).nonzero()[0]
    score = score[chosen]
    bbox = np.asarray([predictions[i]["bbox"] for i in chosen]).reshape(-1, 4)
    bbox = BoxMode.convert(bbox, BoxMode.XYWH_ABS, BoxMode.XYXY_ABS)

    labels = np.asarray([dataset_id_map(predictions[i]["category_id"]) for i in chosen])

    ret.scores = score
    ret.pred_boxes = Boxes(bbox)
    ret.pred_classes = labels

    try:
        ret.pred_masks = [predictions[i]["segmentation"] for i in chosen]
    except KeyError:
        pass
    return ret

In [ ]:
for dic in tqdm.tqdm(dicts):
  img = cv2.imread(dic["file_name"], cv2.IMREAD_COLOR)[:, :, ::-1]
  basename = os.path.basename(dic["file_name"])
  predictions = create_instances(pred_by_image[dic["image_id"]], img.shape[:2])
  vis = Visualizer(img, metadata)
  vis_pred = vis.draw_instance_predictions(predictions).get_image()
  vis = Visualizer(img, metadata)
  vis_gt = vis.draw_dataset_dict(dic).get_image()
  concat = np.concatenate((vis_pred, vis_gt), axis=1)
  cv2.imwrite(os.path.join(output, basename), concat[:, :, ::-1])

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64encode
import cv2

def imshow(name, img):
  """Put frame as <img src="data:image/jpg;base64,...."> """

  js = Javascript('''
  async function showImage(name, image, width, height) {
    img = document.getElementById(name);
    if(img == null) {
      img = document.createElement('img');
      img.id = name;
      document.body.appendChild(img);
    }
    img.src = image;
    img.width = width;
    img.height = height;
  }
  ''')

  height, width = img.shape[:2]

  ret, data = cv2.imencode('.jpg', img)   # compress array of pixels to JPG data
  data = b64encode(data)                  # encode base64
  data = data.decode()                    # convert bytes to string
  data = 'data:image/jpg;base64,' + data  # join header ("data:image/jpg;base64,") and base64 data (JPG)

  display(js)
  eval_js(f'showImage("{name}", "{data}", {width}, {height})')  # run JavaScript code to put image (JPG as string base64) in <img>
                                          # `name` and `data` in needs `" "` to send it as text, not as name of variabe.

In [ ]:
import os
import cv2
import imutils
import time

folder = r'/content/'

VALID_FORMAT = (".JPG", ".JPEG", ".PNG")

for number, filename in enumerate(os.listdir(folder)):
    if filename.upper().endswith(VALID_FORMAT):
        path = os.path.join(folder, filename)

        image = cv2.imread(path)

        frame = imutils.resize(image, width=400)

        number = number % 2
        imshow(f"img{number}", frame)

        time.sleep(1)